In [39]:
import requests
import datetime
import inspect
import json
from typing import(
    TypedDict,
    List,
    Dict,
    Literal,
    Callable,
    Optional,
    Any,
    get_type_hints
)
from openai import OpenAI
from openai.types.chat.chat_completion_message import ChatCompletionMessage
from openai.types.chat.chat_completion_message_tool_call import ChatCompletionMessageToolCall

In [40]:
from dotenv import load_dotenv
import os
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(
    base_url="https://openai.vocareum.com/v1",
    api_key=api_key
)


In [ ]:
class Memory:
    def __init__(self):
        self._messages: List[Dict[str, str]] = []
    
    def add_message(self, 
                    role: Literal['user', 'system', 'assistant', 'tool'], 
                    content: str,
                    tool_calls: dict=dict(),
                    tool_call_id=None)-> None:

        message = {
            "role": role,
            "content": content,
            "tool_calls": tool_calls,
        }

        if role == "tool":
            message = {
                "role": role,
                "content": content,
                "tool_call_id": tool_call_id,
            }

        self._messages.append(message)

    def get_messages(self) -> List[Dict[str, str]]:
        return self._messages

    def last_message(self) -> None:
        if self._messages:
            return self._messages[-1]

    def reset(self) -> None:
        self._messages = []

In [ ]:
class Tool:
    def __init__(self, func:Callable):
        self.func = func
        self.name = func.__name__
        self.description = func.__doc__
        self.argument_types_map = get_type_hints(func)
        self.signature = inspect.signature(func)      # gets the full signature (parameters and defaults) of a function as a Signature object for introspection.
        self.arguments = [
            {
                "name": key, 
                "type": self._infer_json_schema_type(value),
                "required": param.default == inspect.Parameter.empty
            } 
            for key, value in self.argument_types_map.items()
            if (param := self.signature.parameters.get(key))  #walrus operator
        ]

    def dict(self):
        return {
            "type": "function",
            "function": {
                "name": self.name,
                "description": self.description,
                "parallel_tool_calls": False,
                "parameters": {
                    "type": "object",
                    "properties": {
                        argument["name"]: {
                            "type": argument["type"],
                        }
                        for argument in self.arguments
                    },
                    "required": [
                        argument["name"] 
                        for argument in self.arguments 
                        if argument["required"]
                    ],
                    "additionalProperties": False,
                },
                "strict": True
            }
        }

    def __call__(self, *args, **kwargs):
        return self.func(*args, **kwargs)
    
    def _infer_json_schema_type(self, arg_type: Any) -> str:
        if arg_type == bool:
            return "boolean"
        elif arg_type == int:
            return "integer"
        elif arg_type == float:
            return "number"
        elif arg_type == str:
            return "string"
        elif arg_type == list:
            return "array"
        elif arg_type == dict:
            return "object"
        elif arg_type is None:
            return "null"
        elif arg_type == datetime.date or arg_type == datetime.datetime:
            return "string"  # JSON Schema treats dates as strings
        else:
            return "string"  # Default to string if type is unknown

In [43]:
@Tool
def greet(name: str, age: int = 18):
    """Greets the user by name and age"""
    return f"Hello {name}, you are {age} years old!"

# Call the function
print(greet("Pradhum"))

# View the tool schema
print(json.dumps(greet.dict(), indent=2))



Hello Pradhum, you are 18 years old!
{
  "type": "function",
  "function": {
    "name": "greet",
    "description": "Greets the user by name and age",
    "parallel_tool_calls": false,
    "parameters": {
      "type": "object",
      "properties": {
        "name": {
          "type": "string"
        },
        "age": {
          "type": "integer"
        }
      },
      "required": [
        "name"
      ],
      "additionalProperties": false
    },
    "strict": true
  }
}


In [44]:
class StopReactLoopException(Exception):
    """
    Terminates ReAct loop
    """

In [45]:
TERMINATION_MESSAGE = "StopReactLoopException"

In [46]:
def termination()-> str:
    """Terminate the ReAct loop. If the agent thinks there's no further actions to take"""
    return TERMINATION_MESSAGE 

In [ ]:
class Agent:
    """A ReAct AI Agent"""
    
    def __init__(
        self,
        name:str = "Agent", 
        role:str = "Personal Assistant",
        instructions:str = "Help users with any question",
        model:str = "gpt-4o-mini",
        temperature:float = 0.0,
        tools:List[Tool] = [],
    ):  
        self.name = name
        self.role = role
        self.instructions = instructions
        self.model = model
        self.temperature = temperature
        self.client = client
        self.tools = tools
        self.termination_message = TERMINATION_MESSAGE
        self._register_tool(Tool(termination))
        self.tool_map = {t.name:t for t in tools}
        #Maps tool names to tool objects — helps you quickly look up a tool by name.
        self.openai_tools = [t.dict() for t in self.tools] if self.tools else None
        #Converts each tool into OpenAI-compatible schema using dict() method.
        self.memory = Memory()
        self.memory.add_message(
            role="system",
            content=f"You're an AI Agent, your role is {self.role}, " 
                    f"and you need to {self.instructions} "
                    "You can answer multistep questions by sequentially calling functions. "
                    "You follow a pattern of of Thought and Action. "
                    "Create a plan of execution: "
                    "- Use Thought to describe your thoughts about the question you have been asked. "
                    "- Use Action to specify one of the tools available to you. "
                    "When you think it's over call the termination tool. "
                    "Never try to respond directly if the question needs a tool."
                    "The actions you have are the Tools: "
                    f"{self.openai_tools}",
        )


#Takes user inout and starts REact loop.......
    def invoke(self, user_message: str, max_iter:int=3) -> str:
        self.memory.add_message(
            role="user",
            content=user_message,
        )
        try:
            self._react_loop(max_iter)
        except StopReactLoopException as e:
            print(f"Termninated loop with message: '{e}'")
            self._reason()

        return self.memory.last_message()


#Repeated reasoning and tool calling..................
    def _react_loop(self, max_iter:int):
        for i in range(max_iter):
            self._reason()

            ai_message = self._get_completion(
                messages = self.memory.get_messages(),
                tools=self.openai_tools,
            )
            tool_calls = ai_message.tool_calls

            self.memory.add_message(
                role="assistant",
                content=ai_message.content,
                tool_calls=tool_calls,
            )

            if tool_calls:
                self._call_tools(tool_calls)



#Ask LLM to think without the tools..................
    def _reason(self):
        # No tools
        ai_message = self._get_completion(
            messages = self.memory.get_messages(),
        )

        self.memory.add_message(
            role="assistant",
            content=ai_message.content,
            tool_calls=None,
        )

#Executes the tools call returned by LLM..................
    def _call_tools(self, tool_calls:List[ChatCompletionMessageToolCall]):

        for t in tool_calls:
            tool_call_id = t.id                          #Loads Id
            function_name = t.function.name              #Loads Name
            args = json.loads(t.function.arguments)      #Loads Arguments

            callable_tool = self.tool_map[function_name]
            result = callable_tool(**args)
            
            self.memory.add_message(
                role="tool", 
                content=str(result), 
                tool_call_id=tool_call_id
            )
            if result == TERMINATION_MESSAGE:
                raise StopReactLoopException(TERMINATION_MESSAGE)


#Sends messages and tools to the OpenAI................
    def _get_completion(self, messages:List[Dict], tools:List=None)-> ChatCompletionMessage:
        response = self.client.chat.completions.create(
            model=self.model,
            temperature=self.temperature,
            messages=messages,
            tools=tools,
        )
        
        return response.choices[0].message


#Adds tool to the agent...................................
    def _register_tool(self, tool:Tool):
        self.tools.append(tool)



In [48]:
def power(base:float, exponent:float):
    """Exponentatiation: base to the power of exponent"""
    
    return base ** exponent

In [49]:
def sum(number_1:float, number_2:float):
    """Sum / Addition: Add two numbers"""
    
    return number_1 + number_2

In [50]:
funcs = [power, sum]
tools = [Tool(func) for func in funcs]

In [51]:
agent = Agent(
    tools=tools
)

In [52]:
agent.invoke("Whats 2 to the power of 3? Then add 10 to the result")

Termninated loop with message: 'StopReactLoopException'


{'role': 'assistant',
 'content': 'The final result of \\(2\\) to the power of \\(3\\) plus \\(10\\) is \\(18\\). If you have any more questions or need further assistance, feel free to ask!',
 'tool_calls': None}

In [53]:
agent.memory.get_messages()

[{'role': 'system',
  'content': 'You\'re an AI Agent, your role is Personal Assistant, and you need to Help users with any question You can answer multistep questions by sequentially calling functions. You follow a pattern of of Thought and Action. Create a plan of execution: - Use Thought to describe your thoughts about the question you have been asked. - Use Action to specify one of the tools available to you. When you think it\'s over call the termination tool. Never try to respond directly if the question needs a tool.The actions you have are the Tools: [{\'type\': \'function\', \'function\': {\'name\': \'power\', \'description\': \'Exponentatiation: base to the power of exponent\', \'parallel_tool_calls\': False, \'parameters\': {\'type\': \'object\', \'properties\': {\'base\': {\'type\': \'number\'}, \'exponent\': {\'type\': \'number\'}}, \'required\': [\'base\', \'exponent\'], \'additionalProperties\': False}, \'strict\': True}}, {\'type\': \'function\', \'function\': {\'name\'

In [54]:
for tool in tools:
    print(tool.name)

power
sum
termination


In [60]:
print(list(agent.tool_map))

['power', 'sum', 'termination']


In [ ]:

for t in tool_calls:
    tool_call_id = t.id                          #Loads Id
    function_name = t.function.name              #Loads Name
    args = json.loads(t.function.arguments)      #Loads Arguments

    callable_tool = tool_map[function_name]
    result = callable_tool(**args)
            

NameError: name 'tool_calls' is not defined